# Wyrażenia regularne

Zapoznamy się z re - modułem biblioteki standardowej.

<br>


A w szczególności z funkcjami:

*  search i findall


<br>

oraz podstawowymi konstrukcjami syntaktycznymi wyrażeń regularnych.

Wyrażenia regularne (regex) związane są z teorią języków formalnych, a dokładnej języków regularnych, o których można również myśleć jako automatach skończonych (są to abstrakcyjne maszyny przyjmująca pewne wyrazy wtedy, gdy sekwencja symboli wejściowych wywołuje zmiany stanu kończące się w stanie akceptującym).

Powszechnie stosuje się je do wyszukiwania wzorców w danych tekstowych.

https://docs.python.org/3/library/re.html

### Znaki specjalne

|||
|--|:--|
|.|dowolny znak, oprócz \n|
|?|zero lub jedno wystąpienie znaku|
|\*|zero lub więcej wystąpień znaku|
|+|jeden lub więcej wystąpień znaku|
|^|początek ciągu tekstowego|
|$|koniec ciągu tekstowego|
|[]|dowolny znak z listy|
|[^]|żaden z listy|
|{}|liczba wystąpień lub {od, do} itd.|
|A\|B| A lub B, mogące być odrębnymi wyrażeniami|

### Klasy znaków

|||
|--|:--|
|\d|dowolna cyfra od 0 do 9|
|\D|wszystko z wyjątkiem cyfr [^d]|
|\s|spacja|
|\S|wszystko co nie jest spacją|
|\w|a-z, A-Z, cyfry, podkreślenie \_|
|\W|znaki nie \w|
|\b|granica pomiędzy \w i \W|

### Grupowanie

|||
|--|:--|
|()|dopasowywuje cały wzorzec, ale wyodrębnia to co w nawiasie|
|(?P\<key>)|przypisuje nazwę \<key> grupie zadanego wzorca|

Kompletniejsze zestawienie znaków oraz ich różnych funkcji można znaleźć w poniższym dokumencie:

https://www.dataquest.io/wp-content/uploads/2019/03/python-regular-expressions-cheat-sheet.pdf

In [1]:
# raw string (r) traktuje \ jako normalny znak
print(len("\n"), len(r"\n"))

1 2


* re.match: dopasowuje wyrażenie do początku tekstu i zwraca obiekt match lub None

* re.search: skanuje text i dopasowuje wyrażenie w pierwszym możliwym miejscu

* re.findall: zwraca listę krotek z wszystkimi nienachodzącymi dopasowaniami

In [2]:
import re

text = '30-123 Kraków'
expr = '([0-9]{2}\-[0-9]{3})\ ([^\ ]+)$'

match = re.search(expr, text)

if match is None:
    raise Exception('Niepoprawny format')
    
else:
    
    # match.groups() zwraca krotkę z wyodrębnionymi z dopasowania grupami
    zcode, city = match.groups()
    print(f'Kod: {zcode}\nMiasto: {city}')
    
    #match.span() zwraca krotkę z pozycją dopasowanego ciągu znaków
    print(match.span())

Kod: 30-123
Miasto: Kraków
(0, 13)


### Dopasowywanie zachłanne i niezachłanne

In [3]:
txt='HaHaHaHaHaHaHa'

greedy_regex=re.compile('(Ha){1,5}')
result1=greedy_regex.search(txt)

# zwykle ? jest zachłanne, ale tutaj występuje jako modyfikator kwantyfikatora - {m, n}? ignoruje n
nongreedy_regex=re.compile('(Ha){1,5}?')
result2=nongreedy_regex.search(txt)

print(result1)
print(result2)

<re.Match object; span=(0, 10), match='HaHaHaHaHa'>
<re.Match object; span=(0, 2), match='Ha'>


### re.findall

In [4]:
text = '''
ul. Polna 166/10 42-022 Pcim
Uniwersytecka 198/34 81-426 Gdynia
Jagiellońska 126/48 30-654 Kraków
'''
expr = '\ ([0-9]{2}\-[0-9]{3})\ (.+)\n'

matches = re.findall(expr, text)
print(matches)

[('42-022', 'Pcim'), ('81-426', 'Gdynia'), ('30-654', 'Kraków')]


### Nadwanie nazw grupom

In [5]:
expr = '\ (?P<zcode>[0-9]{2}\-' + '[0-9]{3})\ (?P<city>.+)\n'

match = re.search(expr, text)
gdict = match.groupdict()


print(gdict)

{'zcode': '42-022', 'city': 'Pcim'}


### Negative / Positive look[behind|ahead] assertion

* A(?=B) | Dopasuje A tylko gdy następuje po nim wyrażenie B
* A(?!B) | Negative lookahead assertion. Dopasuje A tylko gdy nie ma po nim B
* (?<=B)A | Positive lookbehind assertion. Dopasuje A tylko gdy B jest bezpośrednio po lewej (tylko z maksymalną długością)
* (?<!B)A | Negative lookbehind assertion. Dopasuje A tylko gdy B NIE jest bezpośrednio po lewej (tylko z maksymalną długością)

**Zadanie 1.** Enzym restrykcyjny DsaI rozpoznaje sekwencje CCGCGG, CCGTGG, CCACGG, oraz CCATGG. Napisz wyrażenie regularne umożliwiajace sprawdzenie czy dana sekwencja zawiera jego miejsce cięcia. Rozwiązanie nie powinno zawierać 4 całych sekwencji.

**Zadanie 2.** Napisz program, który dla podanej sekwencji DNA odnajdzie w niej wszystkie otwarte ramki odycztu, a następnie wypisze ich przetłumaczoną zawartość. ORF (*ang. Open reading Frame*) to sekwencja znajdująca się pomiędzy kodonem START i STOP.
Zastosuj wyrażenia regularne. Należy przyjać standardowy kod genetyczny (np. z ćw. 3).

*W pełnym rozwiązaniu należy przeszukać łącznie 6 ramek odczytu:

* 3 możliwe pozycje przesunięcia startu rybosomu ([0:3], [1:4], [2:5])
* obie nici DNA mogą ulec transkrypcji: *dna_seq* (tylko w RNA T --> U) i sekwencja komplementarna do *dna_seq* o odwróconej kolejności nukleotydów (reverse complement)

In [6]:
dna_seq = "AGCCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG"

Oczekiwane wyniki:
 * MLLGSFRLIPKETLIQVAGSSPCNLS
 * M
 * MGMTPRLGLESLLE
 * MTPRLGLESLLE
 
https://github.com/urmi-21/orfipy

**Zadanie 3.** Zapisz wyrażenia regularne odnajdujące w poniższym tekście:
* wszystkie słowa zaczynające się na literę 'm'
* wszystkie wyrazy poprzedzające 'and' (włącznie z 'and')
* wszystkie wyrazy mające 7 znaków niezaczynające się na literę 'f'
* wszystkie słowa kończące się na 'ing' o długości <= 7

In [7]:
txt = """Would to heaven that the reader, emboldened and momentarily ferocious as he reads,
finds his wild and savage path through the desolate marshes of these dark and poisonous pages, 
without disorientation; for, unless he brings in his reading a rigorous logic and a tension of mind 
equal at least to his distrust, the mortal emanations of this book will soak his soul, as water sugar. 
It is not good for everybody to read the pages that follow; some alone will savor this bitter fruit without danger. 
Therefore, timid soul, before penetrating farther into such unexplored heaths, directs your heels back and not forward. 
Listen well to what I say to you: run your heels back and not forward, like the eyes of a son who, 
deviates respectfully from the august contemplation of the maternal side; 
or, rather, as an endless angle of chilly cranes of great meditation, which, during the winter, 
flies powerfully through the silence, with all sails stretched, towards a fixed point of the horizon, 
whence suddenly leaves a strange and strong wind, precursor of the storm. 
""".replace('\n', ' ')